In [1]:
%load_ext autoreload
%autoreload 2

In [41]:
from sklearn.model_selection import train_test_split

from gptchem.data import get_polymer_data
from gptchem.evaluator import PolymerKLDivBenchmark, polymer_string2performance, string_distances, get_inverse_polymer_metrics
from gptchem.formatter import InverseDesignFormatter

from gptchem.plotsettings import *

In [42]:
data = get_polymer_data()

In [43]:
from rdkit import Chem
import pandas as pd
formatter = InverseDesignFormatter(
    representation_column="string",
    property_columns=["deltaGmin"],
    property_names=["adsorption_energy"],
   num_classes=5
)

In [44]:
formatterformatted = formatter(data)

In [45]:
data

,Unnamed: 0,smiles,string,deltaGmin,A2_normalized,deltaGmin_cat,A2_normalized_cat,num_[W],max_[W],num_[Tr],...,[W],[W].1,[Tr],[Tr].1,[Ta],[Ta].1,[R],[R].1,rel_shannon,length
0,0,[W][Ta][Tr][W][W][Ta][Ta][Ta][R][W][Tr][Tr][R]...,W-A-B-W-W-A-A-A-R-W-B-B-R-R-B-R,-7.535286,-0.109726,very large,very small,0.250000,2,0.250000,...,4.0,0.25,4.0,0.25,4.0,0.25,4.0,0.25,0.500000,16
1,1,[R][W][W][R][R][Tr][Tr][Tr][Ta][Ta][Ta][W][W][...,R-W-W-R-R-B-B-B-A-A-A-W-W-A-R-B,-7.270527,0.580595,very large,very large,0.400000,2,0.200000,...,4.0,0.25,4.0,0.25,4.0,0.25,4.0,0.25,0.500000,16
2,2,[Ta][R][Ta][W][Tr][W][Ta][R][Tr][W][Ta][Tr][Tr...,A-R-A-W-B-W-A-R-B-W-A-B-B-R-W-R,-6.416311,0.956320,very large,very large,0.000000,0,1.000000,...,4.0,0.25,4.0,0.25,4.0,0.25,4.0,0.25,0.500000,16
3,3,[W][Ta][R][Ta][Tr][Tr][Tr][W][Ta][W][Tr][R][Ta...,W-A-R-A-B-B-B-W-A-W-B-R-A-W-R-R,-6.684816,1.129924,very large,very large,0.000000,0,0.500000,...,4.0,0.25,4.0,0.25,4.0,0.25,4.0,0.25,0.500000,16
4,4,[R][R][Tr][Tr][W][R][Ta][W][R][W][Ta][Tr][Ta][...,R-R-B-B-W-R-A-W-R-W-A-B-A-A-W-B,-6.606492,-0.496439,very large,very small,0.000000,0,0.333333,...,4.0,0.25,4.0,0.25,4.0,0.25,4.0,0.25,0.500000,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3120,3120,[R][W][Tr][W][W][Tr][Tr][Tr][Ta][W][W][W][Ta][...,R-W-B-W-W-B-B-B-A-W-W-W-A-W-A-A-W-A-R-R-B-B-A-...,-16.952768,0.091738,very small,medium,0.181818,3,0.181818,...,12.0,0.25,12.0,0.25,12.0,0.25,12.0,0.25,0.358104,48
3121,3121,[R][Ta][Ta][R][Ta][R][W][Tr][W][Ta][R][Tr][W][...,R-A-A-R-A-R-W-B-W-A-R-B-W-R-B-R-W-B-A-B-A-B-A-...,-17.129209,0.168408,very small,large,0.285714,3,0.142857,...,12.0,0.25,12.0,0.25,12.0,0.25,12.0,0.25,0.358104,48
3122,3122,[W][W][R][Tr][W][W][Ta][W][R][R][Ta][R][Tr][Ta...,W-W-R-B-W-W-A-W-R-R-A-R-B-A-A-W-R-R-B-B-B-A-R-...,-16.361276,-0.068601,very small,very small,0.222222,2,0.222222,...,12.0,0.25,12.0,0.25,12.0,0.25,12.0,0.25,0.358104,48
3123,3123,[Tr][Ta][Tr][Tr][R][W][Ta][R][W][R][Tr][W][R][...,B-A-B-B-R-W-A-R-W-R-B-W-R-W-R-R-W-B-W-A-A-B-A-...,-14.729411,0.000645,small,small,0.111111,2,0.333333,...,12.0,0.25,12.0,0.25,12.0,0.25,12.0,0.25,0.358104,48


In [46]:
def get_continuos_binned_distance(prediction, bin, bins):
    in_bin = (prediction >= bins[bin][0]) & (prediction < bins[bin][1])
    if in_bin:
        loss = 0
    else:
        # compute the minimum distance to bin
        left_edge_distance = abs(prediction - bins[bin][0])
        right_edge_distance = abs(prediction - bins[bin][1])
        loss = min(left_edge_distance, right_edge_distance)
    return loss

In [47]:
get_continuos_binned_distance(-8, 0, list(pairwise(formatter.bins)))

6.782881119999994

In [49]:
formatter.bins

[-inf,
 -14.782881119999994,
 -12.763202617777779,
 -10.908727195555555,
 -8.870141679999994,
 inf]

In [50]:
from more_itertools import pairwise

In [51]:
train, test = train_test_split(data, test_size=10, random_state=42)

In [30]:
sequences = []
counter = 0
for i, row in test.iterrows():
    sequences.append(row['string'])
    counter += 1
    if counter > 9:
        break

In [31]:
test['smiles'] = sequences

In [38]:
formatted_train = formatter(train)
formatted_test = formatter(test)

In [40]:
get_inverse_polymer_metrics(sequences, formatted_test, formatted_train, bins=list(pairwise(formatter.bins)))

-8.254951 3 [(-inf, -14.782881119999994), (-14.782881119999994, -12.763202617777779), (-12.763202617777779, -10.908727195555555), (-10.908727195555555, -8.870141679999994), (-8.870141679999994, inf)]
-13.618059 0 [(-inf, -14.782881119999994), (-14.782881119999994, -12.763202617777779), (-12.763202617777779, -10.908727195555555), (-10.908727195555555, -8.870141679999994), (-8.870141679999994, inf)]
-7.9201155 4 [(-inf, -14.782881119999994), (-14.782881119999994, -12.763202617777779), (-12.763202617777779, -10.908727195555555), (-10.908727195555555, -8.870141679999994), (-8.870141679999994, inf)]
-8.757621 4 [(-inf, -14.782881119999994), (-14.782881119999994, -12.763202617777779), (-12.763202617777779, -10.908727195555555), (-10.908727195555555, -8.870141679999994), (-8.870141679999994, inf)]
-8.392912 4 [(-inf, -14.782881119999994), (-14.782881119999994, -12.763202617777779), (-12.763202617777779, -10.908727195555555), (-10.908727195555555, -8.870141679999994), (-8.870141679999994, inf)

{'composition_mismatches':       distances  min  max  mean  expected_len  found_len
 0  [0, 0, 0, 0]    0    0   0.0             0          0
 1  [0, 0, 0, 0]    0    0   0.0             0          0
 2  [0, 0, 0, 0]    0    0   0.0             0          0
 3  [0, 0, 0, 0]    0    0   0.0             0          0
 4  [0, 0, 0, 0]    0    0   0.0             0          0
 5  [0, 0, 0, 0]    0    0   0.0             0          0
 6  [0, 0, 0, 0]    0    0   0.0             0          0
 7  [0, 0, 0, 0]    0    0   0.0             0          0
 8  [0, 0, 0, 0]    0    0   0.0             0          0
 9  [0, 0, 0, 0]    0    0   0.0             0          0,
 'summary_composition_mismatches': {'min': 0.0,
  'max': 0.0,
  'mean': 0.0,
  'expected_len': 0.0,
  'found_len': 0.0},
 'losses': [0.6151911566235295,
  1.1648219616747983,
  0,
  0,
  0,
  1.740016320500482,
  0,
  0.21990444927869923,
  0.8846648724413999,
  2.295701363835443],
 'kldiv_score': 0.9178404280382662,
 'valid_smiles_f

In [34]:
sequences

['W-R-A-W-W-A-B-W-A-R-B-A-B-A-R-W-B-R-W-W-R-A-B-A-A-R-W-B-W-W',
 'R-R-A-A-W-A-A-R-A-B-W-A-R-B-A-R-R-B-R-W-B-W-W-B-W-A-R-B-A-R-R-A',
 'A-B-A-A-W-W-A-W-A-B-A-A-B-B-R-R-W-W-A-B-W-W-W-A-R-A-W-W-W-W-B-R-B-B',
 'W-A-B-B-A-R-R-B-B-R-B-B-W-B-W-R-W-R-W-W-A-W-W-R-W-A-B-B-B-W-B-B-R-R',
 'B-W-A-R-B-B-B-B-W-R-B-A-W-A-A-W-W-A-A-A-W-R-W-W-R-R-B-B-W-R-W-B-A-B-W-W',
 'A-W-R-B-R-A-R-A-A-A-R-W-W-A-R-B-R-R-R-W-B-R-B-A-A-R',
 'W-B-W-B-W-B-A-W-R-R-B-A-R-W-B-B-A-A-B-W-B-A-W-W-W-R-W-A',
 'W-W-R-A-B-R-A-W-B-B-R-R-A-A-R-A-R-B-W-B-A-A-A-B',
 'R-A-B-B-W-R-B-B-A-W-R-B-W-R-A-R-R-R-R-B-W-R-W-B-R-A-A-A-R-B-W-A-A-B-R-B',
 'A-A-A-R-B-W-A-W-B-A-A-B-A-A-A-B-A-R-B-W-W-R-W-A-R-B-B-W-R-R-W-W-R-A-B-R']